# Session 10
## Introduction
Today we're going to build:

- Linear Regression
- Logistic Regression
- Ridge Regression 
- Naive Bayes 

using health related data. Struecture codes will be provided. Along the way, we'll play with some fun Python codes. By the end, we'll have a complete model building process.

**Please make sure the downloaded data are in the same folder as listed in the drive so that the data loading process will go smoothly.**

In **introduction**, you will work with a simplified sBP dataset that have been consolidated into one```'TXT'``` file available in the data folder as ```'dataset1.txt'```. Specifically, your goal will be to use this data to predict the systolic blood pressure (sBP) of a patient based on the patient's age. 

Since the target variable here is quantitative, this is a regression problem. To begin, we will start with simplest case. You will fit a simple linear regression with just one feature: ```age```. To make the introduction easy to understand, we will not split train and test data but focus more on the model fitting part.

Before that, however, you need to import the data and get it into the form needed by scikit-learn. This involves creating feature and target variable arrays. Furthermore, since you are going to use only one feature to begin with, you need to do some reshaping using NumPy's ```.reshape()``` method. 

### Import packages

**In the first cell we are going to import packages and classes.
It is a good habit to install packages at the beginning of the codes, but for illustration purpose, we import packages at the beginning of each section.**

In [ ]:
# Data manipulation
import pandas as pd 
import numpy as np 

# Model building
from sklearn.linear_model import LinearRegression
# Visualization
import matplotlib.pyplot as plt 
import seaborn as sns 
from matplotlib import style

# Evaluation metrics
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error 
# import statsmodels.api as sm

from termcolor import colored as cl
# Split train and test sets
from sklearn.model_selection import train_test_split

# Load model package
from sklearn.linear_model import LogisticRegression

# Display the plot in notebook and make plots sorted
%matplotlib inline 

### Simple linear regression

In [ ]:
# Download data to data folder
health_df_1 = pd.read_csv('./data/dataset1.txt', delimiter = ",")

To check what is the data type for load-in data, we can use type() function.

In [ ]:
type(health_df_1)

**To double check whether table501.txt has been loaded succesfully and have a brief view on the data. Always list part of the data and check if they make sense.**

In [ ]:
# Display the first 5 rows of the dataframe
health_df_1.head()

We will need to extract X and Y from the dataset by making a subset of the dataframe

In [ ]:
# Extract X (SBP) and Y (Age)
X = health_df_1.SBP
# print(X)
# To extract colum by colum index
# X = health_df_1.iloc[:,0]
# print(X)
Y = health_df_1.Age

print("Before any preprocessing, the shape of X array is:", X.shape)
# Call .reshape() on x because this array is required to be two-dimensional
# For more information about the reshape function:
# https://numpy.org/doc/stable/reference/generated/numpy.reshape.html
X = X.to_numpy().reshape(-1, 1)
print("Before reshaping, the shape of X array is:", X.shape)
Y = Y.to_numpy()
print("The shape of Y array is:", Y.shape)

In [ ]:
illustration_array = np.array([[1,2,3],[4,5,6]])
print("original shape", illustration_array.shape)
illustration_array_1 = illustration_array.reshape(3, -1)
print("reshaped dim 1", illustration_array_1.shape)
illustration_array_2 = illustration_array.reshape(6, -1)
print("reshaped dim 2", illustration_array_2.shape)

### Exploring the data (data visualization)
As always, it is important to explore your data before building models. 

#### Histogram
Histogram helps us to have a brief idea of the data distribution and monitor outliers.

In [ ]:
# Create a histogram with normal curve
# An "interface" to matplotlib.axes.Axes.hist() method
# Use funtion matplotlib.pyplot.hist(x, alpha=n)to plot histogram
# For more infomation:
# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html
n, bins, patches = plt.hist(x=X, # Dataset to be visualized
                            bins='auto', # Automatically 
                            color='#0504aa', # We chose a specific color here
                            alpha=0.7, # Transparency
                            rwidth=0.85) # the bin width in display
plt.grid(axis='y', alpha=0.75) # Set the distance between horizontal lines and display the horizontal lines
plt.xlabel('SBP') # Label name in the x-axis
plt.ylabel('Count') # Label name in the y-axis
plt.title('Histogram of SBP')# Title name of the main plot
plt.show() # Show the plot in Notebooks

#### Scatter plot
While histogram helps us to have an insight about the distribution of the data, scatter plot helps us to better understand the relationship and overall trend between dependent and independent variables.

In [ ]:
# Create a scatterplot to further visualice the relationship between
# SBP and Age
plt.style.use('seaborn-whitegrid') # Grid setting
plt.scatter(X, Y, color='#0504aa') # Same as the histogram function above cells, but this one creates a scatter plot
plt.xlabel('SBP')
plt.ylabel('Age')
plt.title('Scatter plot of SBP')
plt.show()

As you can see, there is a strongly positive correlation, so a linear regression should be able to capture this trend. Your job is to fit a linear regression and then predict the life expectancy, overlaying these predicted values on the plot to generate a regression line. You will also compute and print the $R^2$ score using sckit-learn's ```.score()``` method.

#### Model building
LinearRegression fits a linear model with coefficients w = (w1, …, wp) to minimize the residual sum of squares between the observed targets in the dataset, and the targets predicted by the linear approximation.

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
# Built linear regression model
# Create model class. Currently the model has not been fitted yet.
slr_model = LinearRegression()
# Fit the model
slr_model.fit(X, Y)
# When we want to retrive some information in the fitted model
# we can use model_name.variableToBeRetrived_
intercept = slr_model.intercept_
slope = slr_model.coef_
print('intercept:', intercept)
print('slope:', slope)
# When we want to use the fitted model for prediction
# use the model_name.predict(test_data) function
y_pred = slr_model.predict(X)
# print('predicted response:', y_pred, sep='\n')
y_pred = intercept + slope * X

- red dots: predicted value
- blue dots: original data
- red line: fitted model

In [ ]:
# Visualize regression
plt.xlabel('SBP', fontsize=15)
plt.ylabel('Age', fontsize=15)
# Prediction
plt.scatter(X, Y)
plt.scatter(X, y_pred)
plt.plot(X, y_pred, c = 'r', linewidth=5, alpha=.5, solid_capstyle='round')
plt.show()

### Choose an evaluation metric
* We then need to compare these predictions with the actual result and measure them in some way.
* This is where the selection of evaluation metric is important. For regression, we measure the distance between the predicted and actual answers in some way. The shorter the distance, the more correct the model is. 
* We cover three common metrics below:
  * `Mean Absolute Error`: which provides a mean score for all the predicted versus actual values as an absolute value 
  * `Means Squared Error`: which provides a mean score for all the predicted versus actual values as a square of the absolute value
  * `R2`: the proportion of the variance in the dependent variable that is predictable from the independent variable(s)

In [ ]:
# Obtain the value of MSE 
MSE = mean_squared_error(Y, y_pred)
print('MSE:', MSE)
# Obtain the value of MAE 
MAE = mean_absolute_error(Y, y_pred)
print('MAE:', MAE)

In [ ]:
# To output the p-value for feature
import statsmodels.api as sm

A p-value is a measure of the probability that an observed difference could have occurred just by random chance. The lower the p-value, the greater the statistical significance of the observed difference.

AIC: https://en.wikipedia.org/wiki/Akaike_information_criterion

BIC: https://en.wikipedia.org/wiki/Bayesian_information_criterion

Statsmodels (a statistics package in Python): https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS.html

In [ ]:
slm = sm.OLS(Y, X)
fitted_slm = slm.fit()
print(fitted_slm.summary())

## Part 1: Multiple linear regression

In **Part 1**, you will work with a slightly more complicated clinical dataset that have been consolidated into one ```'CSV'``` file available in the data folder as ```'smoke.csv'```. Specifically, your goal will be to use this data to predict the effect of mother’s smoking during pregnancy (pack/day) and mother’s age at childbirth (years) on birth weight of their infants (oz).

In this section, you will learn how to examin the basic statistics and visualize the relationship between independent variables and split the dataset into ```train``` and ```test``` sets.

In [ ]:
# Load data
health_df_2 = pd.read_csv('./data/smoke.csv')
# View data structure
health_df_2.head()

In [ ]:
# Pandas has a lot of functionality to assist with exploratory data analysis
# .describe() provide summary statistics on all numeric columns
print(health_df_2.describe())
# Similarly, you can also use this function df_name.info()
print(health_df_2.info())
# we can also see the shape of the data
print("\n The shape of dataset 2 is:", health_df_2.shape)

In [ ]:
# Extract predictors and labels from the dataset
X = health_df_2[['Mother_age', 'Mother_smoking']]
Y = health_df_2['Birth_weight']

In [ ]:
# Id is not needed in training/prediction process
# To simplify the dataset you can drop this column
df_2 = health_df_2.drop('ID', axis=1)

### Pair plot 
A pairs plot allows us to see both distribution of single variables and relationships between two variables. Pair plots are a great method to identify trends for follow-up analysis.

In [ ]:
# We can plot the pairplot of variables
# Set grid
style.use('seaborn-whitegrid')
# Set figure size
plt.rcParams['figure.figsize'] = (20,10)
# Specify dataset to used
sns.pairplot(df_2)
# You can also save the plot
plt.savefig('pairplor_health_df_2.png')
# Display the generated plot
plt.show()

### Train/test split for regression
Train and test sets are vital to ensure that your supervised learning model is able to generalize well to new data. This was true for classification models, and is equally true for linear regression models.

In [ ]:
# Split the data set into trianing and validation
X = df_2.drop("Birth_weight", axis = 1)
Y = df_2["Birth_weight"]
# Set random seed here so that you will get the same result as solution
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

Reference for LinearRegression()

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
# Solution
mlr = LinearRegression()
# Fit the model with training data
mlr.fit(X_train, y_train) 
# Call model prediction on test data
y_pred = mlr.predict(X_test)

In [ ]:
# Evaluate mlr model
# Obtain the value of R square
print(cl("R-Squared :", attrs = ["bold"]),
      mlr.score(X_test.to_numpy(),
                y_test))
# Obtain the value of MSE 
MSE = mean_squared_error(y_test, y_pred)
print(cl("MSE:", attrs = ["bold"]), MSE)
# Obtain the value of MAE 
MAE = mean_absolute_error(y_test, y_pred)
print(cl("MAE:", attrs = ["bold"]), MAE)

We then need to fit the model by calling the OLS object’s fit() method. Ignore the warning about the kurtosis test if it appears, we have only 16 examples in our dataset and the test of the kurtosis is valid only if there are more than 20 examples.

In [ ]:
# Solution
X_train.shape

In [ ]:
mlm = sm.OLS(y_train, X_train)
fitted_mlm = mlm.fit()
print(fitted_mlm.summary())

## Part 2: Binary Logistic Regression
This part is modified from Harvard CS109A's lab 6 notebook:
https://harvard-iacs.github.io/2018-CS109A/labs/lab-6/student/

Linear regression is usually a good baseline model, but since the outcome we're trying to predict only takes values 0 and 1 we'll want to use logistic regression instead of basic linear regression.

In this section, you will use the clinical dataset from Ille-et-Vilaine Study of Oesophageal Cancer
- Cases: 200 men with oesophageal cancer (OC)
- Controls: 775 OC-free men randomly drawn from the same regions.
- Primary Interest: To assess associations between alcohol and tobacco consumptions and oesophageal cancer incidence.


In [ ]:
# Load model package
from sklearn.linear_model import LogisticRegression
# Evaluation metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score

In [ ]:
# Read-in and checking
oesophageal_df = pd.read_csv("data/oesophageal.csv", index_col=0)
# Please examine the first 6 rows of the dataset use method in previous 
# data checking process
oesophageal_df.head() # Leave this line blank for exercise

In [ ]:
# Please view the dataset summary matrix with the method in previous 
# data checking process
oesophageal_df.describe() # Leave this line blank for exercise

### Split train and test sets 
reference: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
# Split the data set into trianing and validation
# Time to practise! Please separate the read-in data
# into training and testing sets
X = oesophageal_df.drop("case", axis = 1) # Leave this line blank for exercise
Y = oesophageal_df["case"] # Leave this line blank for exercise
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0) # Leave this line blank for exercise
# Double check whether you have successfully 
# separate training and testing sets
X_train.head()

In [ ]:
# Visualize your training data
# Leave following cell blank for exercise
style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = (20,10)
sns.pairplot(X_train)
plt.savefig('pairplor_oesophageal_df_X_train.png')
plt.show()

**Model building part**
Logistic regression: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
#‘lbfgs’ solver handles multinomial loss in multiclass problems 
# The “balanced” mode uses the values of y to automatically adjust
# weights inversely proportional to class frequencies in the input 
# data as n_samples / (n_classes * np.bincount(y))
logreg_model = LogisticRegression(class_weight="balanced").fit(X_train, y_train)

In [ ]:
y_preds_train = logreg_model.predict(X_train)
y_preds_test = logreg_model.predict(X_test)

full_logreg_score_train = accuracy_score(y_train, y_preds_train)
full_logreg_score_test = accuracy_score(y_test, y_preds_test)

# Evaluation
print('Training Set Score: {}'.format(full_logreg_score_train))
print('Test Set Score: {}'.format(full_logreg_score_test))

#### We will use a build-in function to calculate precision, recall vs thresholds
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html

In [ ]:
# Probs_y is a 2-D array of the probabilities of 
# class 0 (first column of array) vs class 1 (2nd column in array)
probs_y = logreg_model.predict_proba(X_test)
# Retrieve probability of being 1(in second column of probs_y)
precision, recall, thresholds = precision_recall_curve(y_test, probs_y[:, 1]) 
# Similar as plots in above sections
plt.rcParams['figure.figsize'] = (8,6)
plt.title("Precision-Recall vs Threshold Chart")
# Plot dash lines
plt.plot(thresholds, precision[: -1], "b--", label="Precision")
plt.plot(thresholds, recall[: -1], "r--", label="Recall")
plt.ylabel("Precision, Recall")
plt.xlabel("Threshold")
# Specify legend location and size
plt.legend(loc="lower left",fontsize = "large")
plt.ylim([0,1])
plt.show()

In [ ]:
F1 = 2 * (precision * recall) / (precision + recall)
pr_auc = auc(recall, precision)
print("AUC:", pr_auc)
# Similar as plots in above sections
# plt.title("AUC vs Threshold Chart")
plt.rcParams['figure.figsize'] = (8,6)
plt.title("F1 vs Threshold Chart")
# Plot lines
plt.plot(thresholds, F1[:-1], label="F1")
plt.ylabel("F1")
plt.xlabel("Threshold")
plt.legend(loc="lower left")
plt.ylim([0,1])
plt.show()

## Part 3: Ridge Regression
Ridge regression is a regression technique that is quite similar to unadorned least squares linear regression: simply adding an $\ell_2$ **penalty** on the parameters $\beta$ to the objective function for linear regression yields the objective function for ridge regression.

Our goal is to find an assignment to $\beta$ that minimizes the function

$$f(\beta) = \|X\beta - Y\|_2^2 + \lambda \|\beta\|_2^2,$$

where $\lambda$ is a hyperparameter and, as usual, $X$ is the training data and $Y$ the observations. In practice, we tune $\lambda$ until we find a model that generalizes well to the test data.

In this section, you will work with Gapminder data that we have consolidated into one CSV file available in the workspace as 'gapminder.csv'. Specifically, your goal will be to use this data to predict the life expectancy in a given country based on features such as the country's GDP, fertility rate, and population. As in Chapter 1, the dataset has been preproces

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 

In [ ]:
# For k-fold cv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
# Load model
from sklearn.linear_model import Ridge
# Cross validation
from sklearn.model_selection import cross_validate

In [ ]:
# Load dataset
gm_df = pd.read_pickle("./data/gm_2008.pkl")

In [ ]:
gm_df.head()

## Clean data
The dataset may contain a few unknown values. If that is the case, we can use dataset.dropna() function to do data cleaning.

In [ ]:
print("num of na before cleaning: ", gm_df.isna().sum())
gm_df = gm_df.drop(labels=['Region'], axis='columns')
print("num of na after cleaning: ", gm_df.isna().sum())

To create a cleaner dataset, we will drop some columns and remove NaN values manually

In [ ]:
# We create a subdataset based on the previous one
# Split the data set into trianing and validation
X = gm_df.drop('life', axis='columns').values
y = gm_df['life'].values

A test set should still be held out for final evaluation. 

- A model is trained only using the folds labeled as training data;

- The resulting model is validated on the remaining part of the data (i.e., it is used as a test set to compute a performance measure such as accuracy).

Reference: https://scikit-learn.org/stable/modules/cross_validation.html

In [ ]:
Image(url='https://scikit-learn.org/stable/_images/grid_search_cross_validation.png')

In [ ]:
# we create this function to plot alpha with the error range
# don't need to understand
def display_plot(cv_scores, cv_scores_std, alpha_space):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(alpha_space, cv_scores)

    std_error = cv_scores_std / np.sqrt(10)
    # Try to uncomment this line to see the change of the final plot
#     ax.fill_between(alpha_space, cv_scores + std_error, cv_scores - std_error, alpha=0.2)
    ax.set_ylabel('CV Score +/- Std Error')
    ax.set_xlabel('Alpha')
    ax.axhline(np.max(cv_scores), linestyle='--', color='.5')
    ax.set_xlim([alpha_space[0], alpha_space[-1]])
    ax.set_xscale('log')
    ax.set_title('CV Score and std error region vs Alpha')

In [ ]:
# Split the data into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0) 

Here the defualt score of the cv score is R square value.
Scores can also be specified with different evaluation metrics.
For classification task, it has different default score metrics. 

But overally, the higher the score, the better the model.

https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [ ]:
# Try to test different range of alpha 
# alpha_space = [1e-3, 1e-2, 3e-2, 5e-2, 1e-1, 1, 3, 5, 10]
# Setup the array of alphas and lists to store scores
alpha_space = np.logspace(-4, 0, 50)
ridge_scores = []
ridge_scores_std = []

# Create a ridge regressor: ridge
ridge = Ridge(normalize=True)

# Compute scores over range of alphas
for alpha in alpha_space:
    
    # Specify the alpha value to use: ridge.alhpa
    ridge.alpha = alpha
    
    # Perform 10-fold CV: ridge_cv_scores
    ridge_cv_scores = cross_val_score(ridge, X_train, y_train, cv=10)
    
    # Append the mean of ridge_cv_scores to ridge_scores
    ridge_scores.append(np.mean(ridge_cv_scores))
    
    # Append the std of ridge_cv_scores to ridge_scores_std
    ridge_scores_std.append(np.std(ridge_cv_scores))
    
# Display the plot
display_plot(ridge_scores, ridge_scores_std, alpha_space)

In [ ]:
# Exercise time!
# Let's try with some different value of alphas
# and try to outperform alpha = 0.6
alpha = 0.6

Reference for Ridge regression:https:https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

Reference for Lasso regression:https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

In [ ]:
# Define ridge regression model
ridge_model = Ridge(normalize=True)
ridge_model.alpha = 0.6
# Fit the model
ridge_model.fit(X_train, y_train)

In [ ]:
# Make a prediction
y_pred = ridge_model.predict(X_test)

Model evaluation

In [ ]:
# Obtain the value of MSE 
MSE = mean_squared_error(y_test, y_pred)
print('MSE:', MSE)
# Obtain the value of MAE 
MAE = mean_absolute_error(y_test, y_pred)
print('MAE:', MAE)
R2 = ridge_model.score(X_test, y_test)
print("R2:", R2)

## Part 4: Naive Bayes and optional exercise
reference:

https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/05.05-Naive-Bayes.ipynb#scrollTo=-NJHDvC9xpc-

In [ ]:
# Import dataset stimulation package
from sklearn.datasets import make_blobs
# Model package
from sklearn.naive_bayes import GaussianNB
# Decision boundary
from mlxtend.plotting import plot_decision_regions
import matplotlib.gridspec as gridspec
# Evaluation metrics
from sklearn.metrics import auc
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve

Now we will implement the model on clinical data. In this dataset, we are trying to predic whether the patient have diabetes based on the patient's glucose level and bloodpressure. 

In [ ]:
nb_df = pd.read_csv("./data/NB_dataset.txt", delimiter=',')
nb_df.head()

In [ ]:
# Exercise: try to view the summary table for this dataset

# Solution:
print(nb_df.describe())
print(nb_df.info())

Before feeding the data to the naive Bayes classifier model, we need to do some pre-processing.
Here, we’ll create the x and y variables by taking them from the dataset and using the train_test_split function of scikit-learn to split the data into training and test sets.

In [ ]:
X = nb_df[["glucose", "bloodpressure"]]
y = nb_df["diabetes"]

In [ ]:
# Exercise: try to split the dataset into training and validation set with 
# test_size = 0.25 and random_sate = 1

# Solution:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1) # Leave this line blank for exercise

In [ ]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
y_pred = nb_model.predict(X_test)
y_pred

**Model Evaluation**

Finally, we need to check to see how well our model is performing on the test data. For this, we evaluate our model by finding the accuracy score produced by the model.

In [ ]:
accuracy = accuracy_score(y_pred, y_test)*100
f1 = f1_score(y_test, y_pred)
# Avoid using variables with the same name as a function
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
auc_value = auc(fpr, tpr)

In [ ]:
print("Accuracy: ", accuracy)
print("F1: ", f1)
print("AUC: ", auc_value)

**Optional Exercise:**

- Use the population dataset that we used in Ridge regression section
- We will try to do some feature selection based on the information gain

In [ ]:
gm_df.describe()

In [ ]:
# Split the data set into trianing and validation
# Time to practise! Please separate the read-in data
# into training and testing sets
X = gm_df.drop('life', axis='columns').values
# Change continuous labels to binary labels
gm_df['life'] = np.where(gm_df.life > 65, 1, 0)
y = gm_df['life'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0) # Leave this line blank for exercise

In [ ]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
y_pred = nb_model.predict(X_test)
# Dimensional purpose
y_test = y_test.reshape(-1, 1)

In [ ]:
accuracy = accuracy_score(y_pred, y_test)*100
f1 = f1_score(y_test, y_pred)
# Avoid using variables with the same name as a function
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
auc_value = auc(fpr, tpr)

In [ ]:
print("Accuracy: ", accuracy)
print("F1: ", f1)
print("AUC: ", auc_value)

The model has a bad AUC and accuracy values. We will try to improve the prediction by doing some feature selection based on the training set and use only a subset of important variables as predictors.

Sklearn has the function "mutual_info_classif" that can be used to calculate information gain, similar to what we have seen in lecture. More information on the implementation of this can be found at:

https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html

In [ ]:
from sklearn.feature_selection import mutual_info_classif

In [ ]:
# To see which features we have so far
gm_df.columns

In [ ]:
# Calculate the information gain for each feature in the data set
information = mutual_info_classif(X_train, y_train)
features_name = ['population', 'fertility', 'HIV', 'CO2', 'BMI_male', 'GDP', 'child_mortality']
information_gain = dict(zip(information, features_name))

In [ ]:
# Print out the dictionary we just created, with the most significant features first
for i in sorted(information_gain, reverse = True):
    print(str(i) + ": " + str(information_gain[i]))
# Print the number of features in the dictionary, so that we can verify that there are
# as many features in the dictionary as we selected when we created the 'used_features' array
print(len(information_gain))

In [ ]:
# Extract feature index from the original dataset
selected_features = ['child_mortality','fertility', 'BMI_male', 'GDP', 'HIV', 'CO2']
feature_index_list = []
for feature in selected_features:
    feature_index = gm_df.columns.get_loc(feature)
    feature_index_list.append(feature_index)

In [ ]:
print(feature_index_list)

In [ ]:
# Create a subset of the original dataset
X_train = X_train[:, 1:7]

In [ ]:
# Performce the same operation on the test dataset
X_test = X_test[:, 1:7]

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
y_pred = nb_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_pred, y_test)*100
f1 = f1_score(y_test, y_pred)
# Avoid using variables with the same name as a function
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
auc_value = auc(fpr, tpr)

In [ ]:
print("Accuracy: ", "%.0f%%" % accuracy)
print("F1: ", f1)
print("AUC: ", auc_value)